In [1]:
%pip install transformers datasets sentencepiece accelerate

You should consider upgrading via the '/Users/danilkladnitsky/.pyenv/versions/3.10.4/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments
import torch

# -------------------------------
# Step 1: Define training examples
# -------------------------------
examples = [
    {"input": "为词语“昨天”造句", "output": "昨天我去学校。"},
    {"input": "为词语“明天”造句", "output": "明天我飞上海。"},
    {"input": "为词语“改天”造句", "output": "改天我们一起吃饭。"},
    {"input": "为词语“现在”造句", "output": "现在我正在做作业。"},
    {"input": "为词语“朋友”造句", "output": "我的朋友很喜欢看书。"},
    # Add more training examples for better generalization
]

dataset = Dataset.from_list(examples)

Map: 100%|██████████| 5/5 [00:00<00:00, 1536.60 examples/s]
/var/folders/p9/gl1s91rn2fv_77662_261gb80000gn/T/ipykernel_16926/1654200457.py:54: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
1,60.209000
2,63.760000
3,60.301300
4,63.952600
5,54.345300
6,56.928400
7,59.258200
8,64.266600
9,56.313100
10,53.586500


('./sentence-generator-model/tokenizer_config.json',
 './sentence-generator-model/special_tokens_map.json',
 './sentence-generator-model/spiece.model',
 './sentence-generator-model/added_tokens.json',
 './sentence-generator-model/tokenizer.json')

In [ ]:
# -------------------------------
# Step 2: Load model and tokenizer
# -------------------------------
model_name = "uer/gpt2-chinese-cluecorpussmall"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)



In [ ]:
# -------------------------------
# Step 3: Tokenize the dataset
# -------------------------------
def tokenize(batch):
    inputs = tokenizer(batch["input"], padding="max_length", truncation=True, max_length=64)
    outputs = tokenizer(batch["output"], padding="max_length", truncation=True, max_length=64)
    inputs["labels"] = outputs["input_ids"]
    return inputs

tokenized_dataset = dataset.map(tokenize)


In [ ]:

# -------------------------------
# Step 4: Training arguments
# -------------------------------
training_args = Seq2SeqTrainingArguments(
    output_dir="./checkpoints",
    per_device_train_batch_size=4,
    num_train_epochs=10,
    logging_dir="./logs",
    logging_steps=1,
    save_total_limit=2,
    save_steps=10,
    fp16=torch.cuda.is_available(),
)

# -------------------------------
# Step 5: Trainer setup
# -------------------------------
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
)

trainer.train()

# -------------------------------
# Step 6: Save the final model
# -------------------------------
model.save_pretrained("./sentence-generator-model")
tokenizer.save_pretrained("./sentence-generator-model")

In [8]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("./sentence-generator-model")
tokenizer = AutoTokenizer.from_pretrained("./sentence-generator-model")

prompt = "为词语“明天”造句：<extra_id_0>"
inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(**inputs)
sentence = tokenizer.decode(outputs[0])
print(sentence)

<pad> <extra_id_0>词语</s>
